In [111]:
import pandas as pd
import numpy as np
import os
import json
from datetime import date
dir_train = '../data'

In [112]:
data = pd.read_csv(os.path.join(dir_train, 'train_set.csv'), usecols=range(1,11), parse_dates=['timestamp', 'thread_timestamp'])

In [113]:
data = data[
    data.channel.isin(['career', 'big_data', 'deep_learning', 'hardware', 'kaggle_crackers',
           'lang_python',  'lang_r', 'nlp', 'theory_and_practice', 'welcome', 'bayesian', '_meetings', 'datasets']) & 
    data.main_msg
]

In [114]:
data.head()

,msg_id,user_id,channel,timestamp,text,main_msg,thread_id,thread_timestamp,replies_count,reactions
15251,7002610800,U040HKJE7,_meetings,2015-03-22 11:41:48,я немного заработался на этой неделе и забыл д...,1,7002610800,2015-03-22 11:41:48,0,{}
15252,7002629100,U040HKJE7,_meetings,2015-03-22 11:44:51,"теперь тизер, который я бы хотел обсудить. и п...",1,7002629100,2015-03-22 11:44:51,0,{}
15253,7002632300,U040HKJE7,_meetings,2015-03-22 11:45:23,"оно кому-нибудь надо, интересно? есть деи, мыс...",1,7002632300,2015-03-22 11:45:23,0,{}
15254,7002650700,U040HKJE7,_meetings,2015-03-22 11:48:27,за мной кстати должок перед когнитивистами. я ...,1,7002650700,2015-03-22 11:48:27,0,{}
15255,7002654400,U040HKJE7,_meetings,2015-03-22 11:49:04,...я просто могу выложить пару статей и 3 экра...,1,7002654400,2015-03-22 11:49:04,0,{}


In [115]:
# data_train = data.
date_before = date(2017, 4, 1)
train = data[data['timestamp'] < date_before]
val = data[data['timestamp'] > date_before]

In [116]:
mappings = {
    'career': 0,
    'theory_and_practice': 1,
    'deep_learning': 2,
    'lang_python': 3,
    '_meetings': 4,
    'kaggle_crackers': 5,
    'big_data': 6,
    'lang_r': 7,
    'hardware': 8,
    'nlp': 9,
    'welcome': 10,
    'datasets': 11,
    'bayesian': 12
}

In [117]:
train_data = train[['channel', 'text']].reset_index()[['channel', 'text']]
train_data['channel'] = train_data.channel.map(mappings)
train_data = train_data.sort_values('channel').reset_index()[['channel', 'text']]

In [118]:
val_data = val[['channel', 'text']].reset_index()[['channel', 'text']]
val_data['channel'] = val_data.channel.map(mappings)
val_data = val_data.sort_values('channel').reset_index()[['channel', 'text']]

In [119]:
from fastnumbers import isfloat, isint
train_data = train_data[~train_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]
val_data = val_data[~val_data.text.apply(lambda x: isfloat(x) or isint(x) or len(x) < 20)]

In [120]:
train_text = train_data['text'].astype(str).apply(lambda x: x.to_lower())
train_labels =  np.asarray(train_data['channel'], dtype='int8')

val_text = val_data['text'].astype(str).apply(lambda x: x.lower())
val_labels = np.asarray(val_data['channel'], dtype='int8')

In [176]:
train_text = train_text\
    .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x))\
    .apply(lambda x: re.sub('\s+', ' ', x))
    
val_text = val_text\
    .apply(lambda x: re.sub('(<\S+>:?)|(\s?:\S+:\s?)|(&gt;)|([\w\.]*@[\w\.]*)', ' ', x))\
    .apply(lambda x: re.sub('\s+', ' ', x))

In [76]:
import os
import numpy as np

import zipfile

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# parameters initialization
VALIDATION_SPLIT = 0.1
RANDOM_SEED = 42
    
# initialize dictionary size and maximum sentence length
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 400

In [125]:
import string
from keras.preprocessing.sequence import pad_sequences

rus_alphabet = ['а','б','в','г','д','е','ё','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я']

def create_vocab_set():
    alphabet = (list(string.ascii_lowercase) + rus_alphabet + list(string.digits) + list(string.punctuation) + [' ', '\n'])
    vocab_size = len(alphabet)
    vocab = {}
    for ix, t in enumerate(alphabet):
        vocab[t] = ix+1
    return vocab, vocab_size

def text2sequence(text, vocab):
    temp = []
    for review in text:
        temp.append([])
        for i in review:
            char = vocab.get(i,0)
            if char != 0:
                temp[-1].append(char)
    return temp

vocab, vocab_size = create_vocab_set()

X_train = text2sequence(train_text, vocab)
X_val = text2sequence(val_text, vocab)

X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, value=0)
X_val = pad_sequences(X_val, maxlen=MAX_SEQUENCE_LENGTH, value=0)

In [61]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels, num_classes=13)
val_labels = to_categorical(val_labels, num_classes=13)

In [62]:
train_labels.shape

(100693, 13)

In [63]:
import tensorflow as tf
# ohe функция
def ohe(x, sz):
    return tf.to_float(tf.one_hot(x, sz, on_value=1, off_value=0, axis=-1))

In [64]:
from keras.models import Model
from keras.layers import Input, Lambda
from keras.layers import MaxPooling1D, LSTM, Conv1D, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

# callbacks initialization
# automatic generation of learning curves
callback_1 = TensorBoard(log_dir='./logs/logs_{}'.format(NAME), histogram_freq=0,
                             write_graph=False, write_images=False)
# stop training model if accuracy does not increase more than five epochs
callback_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
# best model saving
callback_3 = ModelCheckpoint("../models/model_{}.hdf5".format(NAME), monitor='val_acc',
                                 save_best_only=True, verbose=1)

NAME = "char_cnn_ohe"
# инициализация входа
in_sentence = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int64')
# Lambda слой для ohe преобразования
embedded = Lambda(ohe, output_shape=lambda x: (x[0], x[1], vocab_size), arguments={"sz": vocab_size})(in_sentence)
block = embedded
# свертки с MaxPooling
for i in range(3):
    block = Conv1D(activation="relu", filters=100, kernel_size=4, padding="valid")(block)
    if i == 0:
        block = MaxPooling1D(pool_size=5)(block)
# LSTM ячейка
block = LSTM(128, dropout=0.1, recurrent_dropout=0.1)(block)
block = Dense(100, activation='relu')(block)
block = Dense(13, activation='softmax')(block)
# собираем модель
model = Model(inputs=in_sentence, outputs=block)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.summary()
model.fit(X_train, train_labels, validation_data=[X_val, val_labels],
         batch_size=1024, epochs=100, callbacks=[callback1, callback2, callback3])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 400)               0         
_________________________________________________________________
lambda_3 (Lambda)            (None, 400, 70)           0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 397, 100)          28100     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 79, 100)           0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 76, 100)           40100     
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 73, 100)           40100     
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
__________

KeyboardInterrupt: 